# HackUMBC: DeFi Credibility 

Cryptocurrency users are constantly plagued by various scams and attacks on their holdings. With a lack of regulation, attackers have been able to exploit protocols which has resulted in over $10 billion lost due to theft and scams.

This notebook analyzes different cryptocurrency wallet addresses in order to determine their credibility for peer-to-peer transactions. By providing users a credibility score, we are able to assist consumers with a risk assessment of the parties they are transacting with. 

In [1]:
!cd hackUMBC2022 && make install

pip install --upgrade pip &&\
pip install -r requirements.txt
^C
ERROR: Operation cancelled by user
make: *** [Makefile:2: install] Interrupt


In [2]:
import pandas as pd
import scipy
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

## API: Gathering ETH Transactions

Through the use of Covalent API, we get the transactions of the Alpha Homora V2 Exploiter's address.

Attack Post-Mortem: https://medium.com/cream-finance/alpha-homora-v2-exploit-post-mortem-344d277bdea6

In [3]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()
# api_key = os.environ.get("COVALENT_KEY")
api_key = os.getenv("COVALENT_KEY")

# address = "0x905315602Ed9a854e325F692FF82F58799BEaB57"
BASE_URL = "https://api.covalenthq.com"


def get_transactions(address) -> requests.Response:
    """Get transactions of a given ETH address"""
    transactions_endpoint = (
        BASE_URL + f"/v1/1/address/{address}/transactions_v2/?key={api_key}"
    )
    params = {
        "quote-currency": "USD",
        "no-logs": True
    }
    transactions = requests.get(transactions_endpoint, params=params, timeout=15)
    return transactions

In [4]:
import json

def main(address="0x905315602ed9a854e325f692ff82f58799beab57"):
    """Driver code"""
    transaction = get_transactions(address)
    with open("transaction.json", "w", encoding="utf-8") as outfile:
        json.dump(transaction.json(), outfile, indent = 4, sort_keys=True)


if __name__ == "__main__":
    # ref: https://etherscan.io/apis
    main()


## Data Cleaning

Here, we convert the .json file captured by the API to .csv 

In [5]:
with open("transaction.json", "r") as f:
    transactions = json.load(f)
transaction_array = transactions['data']['items']
transaction_df = pd.DataFrame(transaction_array)
transaction_df.to_csv('transaction.csv')
transaction_df.drop(['from_address_label', 'to_address_label'], axis=1)

,block_height,block_signed_at,fees_paid,from_address,gas_offered,gas_price,gas_quote,gas_quote_rate,gas_spent,successful,to_address,tx_hash,tx_offset,value,value_quote
0,15631798,2022-09-28T12:20:47Z,34605344503815264,0xdf120a3e36ba6bf32752e47d2d827c1ddd848bab,8000888,15059152224,45.551574,1316.316162,2297961,True,0x14d1b27d79e97e96622618f9d4fa9b1e1e9ef082,0xada3aec40a63b429bab9db1bb4cd11618ecc63ecaaef...,187,0,0.000000
1,15580822,2022-09-21T09:17:23Z,10594727500000000,0x7956f86dacd93b9a6a7aac01f3d86e3c9a851bed,3027065,3500000000,13.512450,1275.393799,3027065,True,0x5d80a8d8cb80696073e82407968600a37e1dd780,0x12cecde19750bdca395ae388e86b0f1cee24410d4428...,476,0,0.000000
2,15131691,2022-07-13T02:42:05Z,56685569282692673,0x905315602ed9a854e325f692ff82f58799beab57,5500000,13492416803,60.433169,1066.112061,4201291,True,None,0x629ccd0b003c9b2b62379c0f0c3dda9050a292b705db...,116,0,0.000000
3,14273802,2022-02-25T06:57:31Z,186024440000000000,0x825a8ba45aa8041d642aa01d857caee17c0d9e3e,5575964,38000000000,503.213323,2705.092529,4895380,True,0xcec8f07014d889442d7cf3b477b8f72f8179ea09,0x58562f741db401d48753b36d73ba03b59ac386fd4cbe...,52,15000000000000000,40.576388
4,14168574,2022-02-09T00:12:55Z,3128557039441680,0xd690758a425ed7b2e3a225c2b40bc65300c67d4f,25080,124743103646,10.097618,3227.563965,25080,True,0x905315602ed9a854e325f692ff82f58799beab57,0xe586fca00e4cde4fdee91052afc8c3329cfc9f45bdfd...,6,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,11846608,2021-02-13T06:05:57Z,None,0x905315602ed9a854e325f692ff82f58799beab57,794084,200000000000,195.522611,1826.658569,535192,True,0x5f5cd91070960d13ee549c9cc47e7a4cd00457bb,0xf31ee9d9e83db3592601b854fe4f8b872cecd0ea2a32...,194,0,0.000000
65,11846605,2021-02-13T06:04:32Z,None,0x905315602ed9a854e325f692ff82f58799beab57,1476660,200000000000,383.984455,1826.658569,1051057,True,0x5f5cd91070960d13ee549c9cc47e7a4cd00457bb,0xcc57ac77dc3953de7832162ea4cd925970e064ead3f6...,12,0,0.000000
66,11846579,2021-02-13T05:59:41Z,None,0x905315602ed9a854e325f692ff82f58799beab57,1505110,125000000000,273.717708,1826.658569,1198769,True,0x560a8e3b79d23b0a525e15c6f3486c6a293ddad2,0x4441eefe434fbef9d9b3acb169e35eb7b3958763b74c...,33,1500000000000000000,2739.987854
67,11846504,2021-02-13T05:40:24Z,None,0x905315602ed9a854e325f692ff82f58799beab57,1293094,128920000000,302.882626,1826.658569,1286165,False,0x560a8e3b79d23b0a525e15c6f3486c6a293ddad2,0x282be26ee4154603cc8f824ce4ace2e0fca850962394...,129,1500000000000000000,2739.987854


## Data Visualization

In [6]:
_deepnote_run_altair(transaction_df, """{"data":{"name":"placeholder"},"mark":{"type":"bar","tooltip":true},"height":220,"$schema":"https://vega.github.io/schema/vega-lite/v4.json","autosize":{"type":"fit"},"encoding":{"x":{"sort":null,"type":"nominal","field":"to_address","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"COUNT(*)","scale":{"type":"linear","zero":true},"aggregate":"count"},"color":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [7]:
_deepnote_run_altair(transaction_df, """{"data":{"name":"placeholder"},"mark":{"type":"bar","tooltip":true},"height":220,"$schema":"https://vega.github.io/schema/vega-lite/v4.json","autosize":{"type":"fit"},"encoding":{"x":{"sort":null,"type":"nominal","field":"from_address","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"COUNT(*)","scale":{"type":"linear","zero":true},"aggregate":"count"},"color":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4beb2dee-7648-4358-9d54-16785e02bd6c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>